In [2]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

cnn_model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(12, (3, 3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10)])

cnn_model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
cnn_model.summary()
cnn_model.fit(x_train, y_train, epochs=1, validation_split=0.1)
cnn_model.evaluate(x_test,  y_test)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 12)        120       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 12)        0         
_________________________________________________________________
flatten (Flatten)            (None, 2028)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                20290     
Total params: 20,410
Trainable params: 20,410
Non-trainable params: 0
_________________________________________________________________
313/313 [==============================] - 0s 1ms/step - loss: 0.1149 - accuracy: 0.9690


[0.1149488165974617, 0.968999981880188]

In [15]:
converter = tf.lite.TFLiteConverter.from_keras_model(cnn_model)
cnn_tflite_model = converter.convert()

with open("./cnn_model.tflite", 'wb') as f:
    f.write(cnn_tflite_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: C:\Users\lswsj\AppData\Local\Temp\tmp3hci0h46\assets


In [10]:
import tensorflow_model_optimization as tfmot
import numpy as np

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = x_train.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

pruned_model = tfmot.sparsity.keras.prune_low_magnitude(cnn_model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
pruned_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

pruned_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_conv2d ( (None, 26, 26, 12)        230       
_________________________________________________________________
prune_low_magnitude_max_pool (None, 13, 13, 12)        1         
_________________________________________________________________
prune_low_magnitude_flatten  (None, 2028)              1         
_________________________________________________________________
prune_low_magnitude_dense (P (None, 10)                40572     
Total params: 40,804
Trainable params: 20,410
Non-trainable params: 20,394
_________________________________________________________________


In [11]:
callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep()
]

pruned_model.fit(x_train, y_train,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/2
422/422 [==============================] - 5s 13ms/step - loss: 0.0994 - accuracy: 0.9718 - val_loss: 0.0785 - val_accuracy: 0.9798
Epoch 2/2
422/422 [==============================] - 5s 12ms/step - loss: 0.0935 - accuracy: 0.9728 - val_loss: 0.0779 - val_accuracy: 0.9793


In [17]:
pruned_model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.0979 - accuracy: 0.9713


[0.09786249697208405, 0.9713000059127808]

In [18]:
pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model)
pruned_cnn_tflite_model = converter.convert()

with open("./pruned_cnn_model.tflite", 'wb') as f:
    f.write(pruned_cnn_tflite_model)

INFO:tensorflow:Assets written to: C:\Users\lswsj\AppData\Local\Temp\tmpsmt5nbnb\assets
INFO:tensorflow:Assets written to: C:\Users\lswsj\AppData\Local\Temp\tmpsmt5nbnb\assets


In [19]:
display(os.path.getsize("cnn_model.tflite"))
display(os.path.getsize("pruned_cnn_model.tflite"))
display(get_gzipped_model_size("cnn_model.tflite"))
display(get_gzipped_model_size("pruned_cnn_model.tflite"))

83432

83432

In [13]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [14]:
get_gzipped_model_size(cnn_model)

TypeError: stat: path should be string, bytes, os.PathLike or integer, not Sequential

In [ ]:
import numpy as np

def run_tflite_model(path, x_test, y_test):
    interpreter = tf.lite.Interpreter(str(path))
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    y_pred = []
    for i, test_image in enumerate(x_test):
        test_image = np.expand_dims(test_image, axis=0).astype(np.float32)

        interpreter.set_tensor(input_details['index'], test_image)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details['index'])

        y_pred.append(output.argmax())

    y_pred = np.array(y_pred)
    accuracy = (y_pred == y_test).mean()
    return accuracy
